# Regresión Logística: Detección de SPAM

En este ejercicio de muestran los fundamentos de la regresión logística plantenado uno de los primeros problemas que fueron solucionados mediante el uso de técnicas de machine learning: La detección de SPAM.

## Enunciado del Ejercicio

Se propone la contrucción de un sistema de aprendizaje automático capaz de predecir si un correo deteminado se corresponde con un correo de SPAM o no, para ello, se utilizará el siguiente conjunto de datos:

##### [2007 TREC Public Spam Corpus](https://terabox.com/s/1cW1v51vsnazpI2PjXAXvCA)
The corpus trec07p contains 75, 419 messages:

    25,220 ham
    50,199 spam
These messages consitute all the messages delivered to a particular server between these dates:

    Sun, 8 Apr 2007 13:07:21 -0400
    Fri, 6 Jul 2007 07:04:53 -0400

# 1. Funciones Complementarias

En este caso práctico relacionado con la detección de correos electrónicos de SPAM, el conjunto de datos que se dispone, está formado por correos eléctronicos, con sus correspondientes encabezados y campos adicionales. Por tanto, se requiere de un procesamiento previo antes de ser ingeridos por el algoritmo de Machine Learning.

In [1]:

# Esta clase facilita el preprocesamiento de correos eléctronicos
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        #charrefs - referencia a 
        self.convert_charrefs = True
        self.fed = []
        
    def handle_data(self, d):
        self.fed.append(d)
        
    def get_data(self):
        return ''.join(self.fed)

In [2]:
# Esta funcion se encarga de eliminar los tags HTML que se encuentren en el texto del correo.
#strip-separar
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()


In [3]:
# Ejemplo de eliminación de los tags HTML de un texto
t = '<tr><td aling="left"><a href="../../issues/51/16.html#article">Phrack World News</a></td>'
strip_tags(t)

'Phrack World News'

Además de eliminar los posibles tags HTML que se encuentren en el correo electrónico, deben realizarse otras opciones de preprocesamiento para evitar que los mensajes contengan ruido inecesario. Entre ellas se encuentra la eliminación de los signos de puntuación, eliminación de posibles campos de correo electrónico que no son relevantes o eliminacióm de los afijos de una palabra manteniendo únicamente la raíz de la misma (Stemming). La clase que se muestra a continuación realiza estas transformaciones.

In [4]:
import email
import string
import nltk

class Parser:
    
    def __init__(self):S
        self.stemmer = nltk.PorterStemmer()
        self.stopwords = set(nltk.corpus.stopwords.words('english'))
        self.punctuation = list(string.punctuation)
        
    def parse(self, email_path):
        """Parse an email."""
        with open(email_path, errors='ignore') as e:
            msg = email.message_from_file(e)
        return None if not msg else self.get_email_content(msg)
    
    def get_email_content(self, msg):
        """Extract the email content."""
        subject = self.tokenize(msg['Subject']) if msg['Subject'] else []
        body = self.get_email_body(msg.get_payload(),
                                  msg.get_content_type())
        content_type = msg.get_content_type()
        # Returning the content of the email
        return {"subject": subject,
               "body": body,
               "content_type": content_type}
    
    def get_email_body(self, payload, content_type):
        """Extract the body of the email."""
        body = []
        if type(payload) is str and content_type == 'text/plain':
            return self.tokenize(payload)
        elif type(payload) is str and content_type == 'text/html':
            return self.tokenize(strip_tags(payload))
        elif type(payload) is list:
            for p in payload:
                body += self.get_email_body(p.get_payload(),
                                           p.get_content_type())
        return body
    
    def tokenize(self, text):
        """Transform a text string in tokens. Perform two main actions, 
        clean the punctuation symbols and stemming of the text"""
        # steamming-Elimiación de los afijos y caracteres especiales
        for c in self.punctuation:
            text = text.replace(c, " ")
        text = text.replace("\t", " ")
        text = text.replace("\n", " ")
        # Split -> dividir
        tokens = list(filter(None, text.split(" ")))
        # Stemming of the tokens
        return[self.stemmer.stem(w) for w in tokens if w not in self.stopwords]

### Lectura de un correo en formato raw

In [5]:
inmail = open("datasets/trec07p/data/inmail.1").read()
print(inmail)

From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007
Return-Path: <RickyAmes@aol.com>
Received: from 129.97.78.23 ([211.202.101.74])
	by speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;
	Sun, 8 Apr 2007 13:07:21 -0400
Received: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100
Message-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>
From: "Tomas Jacobs" <RickyAmes@aol.com>
Reply-To: "Tomas Jacobs" <RickyAmes@aol.com>
To: the00@speedy.uwaterloo.ca
Subject: Generic Cialis, branded quality@ 
Date: Sun, 08 Apr 2007 21:00:48 +0300
X-Mailer: Microsoft Outlook Express 6.00.2600.0000
MIME-Version: 1.0
Content-Type: multipart/alternative;
	boundary="--8896484051606557286"
X-Priority: 3
X-MSMail-Priority: Normal
Status: RO
Content-Length: 988
Lines: 24

----8896484051606557286
Content-Type: text/html;
Content-Transfer-Encoding: 7Bit

<html>
<body bgcolor="#ffffff">
<div style="border-color: #00FFFF; border-right-width: 0px; border-bottom-width: 0px; margin-bottom: 0px;" align="

##### Parsin del correo electrónico

In [6]:
p = Parser()
p.parse("datasets/trec07p/data/inmail.1")

{'subject': ['gener', 'ciali', 'brand', 'qualiti'],
 'body': ['do',
  'feel',
  'pressur',
  'perform',
  'rise',
  'occas',
  'tri',
  'viagra',
  'anxieti',
  'thing',
  'past',
  'back',
  'old',
  'self'],
 'content_type': 'multipart/alternative'}

##### Lectura del índice

Estas funciones complementarias se encargan de cargar en memoria la ruta de cada uno de los correos eléctronicos y su etiqueta correspondiente {spam, ham}.

In [7]:
index = open("datasets/trec07p/full/index").readlines()
index

['spam ../data/inmail.1\n',
 'ham ../data/inmail.2\n',
 'spam ../data/inmail.3\n',
 'spam ../data/inmail.4\n',
 'spam ../data/inmail.5\n',
 'spam ../data/inmail.6\n',
 'spam ../data/inmail.7\n',
 'spam ../data/inmail.8\n',
 'spam ../data/inmail.9\n',
 'ham ../data/inmail.10\n',
 'spam ../data/inmail.11\n',
 'spam ../data/inmail.12\n',
 'spam ../data/inmail.13\n',
 'spam ../data/inmail.14\n',
 'spam ../data/inmail.15\n',
 'spam ../data/inmail.16\n',
 'spam ../data/inmail.17\n',
 'spam ../data/inmail.18\n',
 'spam ../data/inmail.19\n',
 'ham ../data/inmail.20\n',
 'ham ../data/inmail.21\n',
 'spam ../data/inmail.22\n',
 'spam ../data/inmail.23\n',
 'spam ../data/inmail.24\n',
 'spam ../data/inmail.25\n',
 'spam ../data/inmail.26\n',
 'spam ../data/inmail.27\n',
 'spam ../data/inmail.28\n',
 'ham ../data/inmail.29\n',
 'spam ../data/inmail.30\n',
 'ham ../data/inmail.31\n',
 'spam ../data/inmail.32\n',
 'spam ../data/inmail.33\n',
 'ham ../data/inmail.34\n',
 'spam ../data/inmail.35\n',
 

In [8]:
import os
DATASET_PATH = "datasets/trec07p"

def parse_index(path_to_index, n_elements):
    ret_indexes = []
    index = open(path_to_index).readlines()
    for i in range(n_elements):
        mail = index[i].split(" ../")
        label = mail[0]
        path = mail[1][:-1]
        ret_indexes.append({"label": label, "email_path": os.path.join(DATASET_PATH, path)})
    return ret_indexes

In [9]:
def parse_email(index):
    p = Parser()
    pmail = p.parse(index["email_path"])
    return pmail, index["label"]

In [10]:
indexes = parse_index("datasets/trec07p/full/index",1000)
indexes

[{'label': 'spam', 'email_path': 'datasets/trec07p/data/inmail.1'},
 {'label': 'ham', 'email_path': 'datasets/trec07p/data/inmail.2'},
 {'label': 'spam', 'email_path': 'datasets/trec07p/data/inmail.3'},
 {'label': 'spam', 'email_path': 'datasets/trec07p/data/inmail.4'},
 {'label': 'spam', 'email_path': 'datasets/trec07p/data/inmail.5'},
 {'label': 'spam', 'email_path': 'datasets/trec07p/data/inmail.6'},
 {'label': 'spam', 'email_path': 'datasets/trec07p/data/inmail.7'},
 {'label': 'spam', 'email_path': 'datasets/trec07p/data/inmail.8'},
 {'label': 'spam', 'email_path': 'datasets/trec07p/data/inmail.9'},
 {'label': 'ham', 'email_path': 'datasets/trec07p/data/inmail.10'},
 {'label': 'spam', 'email_path': 'datasets/trec07p/data/inmail.11'},
 {'label': 'spam', 'email_path': 'datasets/trec07p/data/inmail.12'},
 {'label': 'spam', 'email_path': 'datasets/trec07p/data/inmail.13'},
 {'label': 'spam', 'email_path': 'datasets/trec07p/data/inmail.14'},
 {'label': 'spam', 'email_path': 'datasets/tr

## 2.- Preprocesamiento del conjunto de datos. 

Con las funciones anteriores se permite la lectura de los correos electrónicos de manera programática el procesamiento de los mismos para eliminar aquellos componentes que no resultan de utilidad para la derección de correos de SPAM. Sin embargo, cada uno de los correos sigue estando representado por un diccionario de Python con una serie de palabras

In [11]:
# Cargar el índice y las etiquetas en memoria
index = parse_index("datasets/trec07p/full/index",1)

In [12]:
# Leer el primer correo
import os
open(index[0]["email_path"]).read()

'From RickyAmes@aol.com  Sun Apr  8 13:07:32 2007\nReturn-Path: <RickyAmes@aol.com>\nReceived: from 129.97.78.23 ([211.202.101.74])\n\tby speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;\n\tSun, 8 Apr 2007 13:07:21 -0400\nReceived: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100\nMessage-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>\nFrom: "Tomas Jacobs" <RickyAmes@aol.com>\nReply-To: "Tomas Jacobs" <RickyAmes@aol.com>\nTo: the00@speedy.uwaterloo.ca\nSubject: Generic Cialis, branded quality@ \nDate: Sun, 08 Apr 2007 21:00:48 +0300\nX-Mailer: Microsoft Outlook Express 6.00.2600.0000\nMIME-Version: 1.0\nContent-Type: multipart/alternative;\n\tboundary="--8896484051606557286"\nX-Priority: 3\nX-MSMail-Priority: Normal\nStatus: RO\nContent-Length: 988\nLines: 24\n\n----8896484051606557286\nContent-Type: text/html;\nContent-Transfer-Encoding: 7Bit\n\n<html>\n<body bgcolor="#ffffff">\n<div style="border-color: #00FFFF; border-right-width: 0px; border-bottom-width: 0

In [13]:
# Parsear el primer correo
mail, label = parse_email(index[0])
print("El correo es: ", label)
print(mail)

El correo es:  spam
{'subject': ['gener', 'ciali', 'brand', 'qualiti'], 'body': ['do', 'feel', 'pressur', 'perform', 'rise', 'occas', 'tri', 'viagra', 'anxieti', 'thing', 'past', 'back', 'old', 'self'], 'content_type': 'multipart/alternative'}


El algoritmo de regresión logística no es capaz de ingerir texto como parte del conjunto de datos, por lo tanto deben aplicarse a una serie de funciones adicionales que transformen el texto de los correos electrónicos parseados en una representación numérica.

#### Aplicación con CountVectorizer

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
# Preparar el email en una cadena de texto
prep_email = [" ".join(mail['subject']) + " ".join(mail['body'])]

vectorizer = CountVectorizer()
# fit = ajustar
x = vectorizer.fit(prep_email)

print("email: ",prep_email, "\n")
print("Caracteristicas de entrada: ",vectorizer.get_feature_names_out())

email:  ['gener ciali brand qualitido feel pressur perform rise occas tri viagra anxieti thing past back old self'] 

Caracteristicas de entrada:  ['anxieti' 'back' 'brand' 'ciali' 'feel' 'gener' 'occas' 'old' 'past'
 'perform' 'pressur' 'qualitido' 'rise' 'self' 'thing' 'tri' 'viagra']


In [15]:
x = vectorizer.transform(prep_email)
print("\nValues: \n", x.toarray())


Values: 
 [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


#### Aplicación de OneHotEncoding

In [16]:
from sklearn.preprocessing import OneHotEncoder

prep_email = [[w] for w in mail['subject'] + mail['body']]

enc = OneHotEncoder(handle_unknown = 'ignore')
x = enc.fit_transform(prep_email)

print("Features: \n", enc.get_feature_names_out())
print("\nValues: \n", x.toarray())

Features: 
 ['x0_anxieti' 'x0_back' 'x0_brand' 'x0_ciali' 'x0_do' 'x0_feel' 'x0_gener'
 'x0_occas' 'x0_old' 'x0_past' 'x0_perform' 'x0_pressur' 'x0_qualiti'
 'x0_rise' 'x0_self' 'x0_thing' 'x0_tri' 'x0_viagra']

Values: 
 [[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [17]:
def create_prep_dataset(index_path, n_elements):
    x = []
    y = []
    indexes = parse_index(index_path, n_elements)
    for i in range(n_elements):
        print("\rParsing email: {0}".format(i+1), end = '')
        mail, label = parse_email(indexes[i])
        x.append("".join(mail['subject']) + " ".join(mail['body']))
        y.append(label)
    return x, y
    

### 3. Entrenamiento del algoritmo

In [18]:
# Leer únicamente un subconjunto de 100 correos electrónicos 
x_train, y_train = create_prep_dataset("datasets/trec07p/full/index", 100)
x_train

Parsing email: 100

['genercialibrandqualitido feel pressur perform rise occas tri viagra anxieti thing past back old self',
 'typodebianreadmhi updat gulu i check mirror it seem littl typo debian readm file exampl http gulu usherbrook ca debian readm ftp ftp fr debian org debian readm test lenni access releas dist test the current test develop snapshot name etch packag test unstabl pass autom test propog releas etch replac lenni like readm html yan morin consult en logiciel libr yan morin savoirfairelinux com 514 994 1556 to unsubscrib email debian mirror request list debian org subject unsubscrib troubl contact listmast list debian org',
 'authentviagramega authenticv i a g r a discount pricec i a l i s discount pricedo miss it click http www moujsjkhchum com authent viagra mega authenticv i a g r a discount pricec i a l i s discount pricedo miss it click',
 'nicetalkyahey billi realli fun go night talk said felt insecur manhood i notic toilet quit small area worri websit i tell secret weapon extra 3 in

#### Aplicar la vectorización a los datos

In [19]:
vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(x_train)

In [20]:
print(x_train.toarray())
print("\nFeatures: ",len(vectorizer.get_feature_names_out()))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

Features:  4621


In [21]:
import pandas as pd
pd.DataFrame(x_train.toarray(), columns = [vectorizer.get_feature_names_out()])

,00,000,0000,000000,000099,0001pt,000m,0085,009,01000u,...,ҳϊ,ӧanz,ӭѯ,ԡšݡ淶,լһʽ,չҵϣ,سŵþʊʊݾѯ,ڶҵţ,㶫иï26,饻jwk
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
y_train

['spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'ham',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'ham',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'ham',
 'ham',
 'spam',
 'spam',
 'spam',
 'ham',
 'ham',
 'ham',
 'spam',
 'ham',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'ham',
 'spam',
 'ham',
 'spam',
 'spam',
 'ham',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam',
 'spam']

##### Entrenamiento del algoritmo de regresión logística con el conjunto de datos preprocesado.

In [23]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(x_train, y_train)

LogisticRegression()

### 4.- Predicción.

##### Lectura de un conjunto de correos nuevos. 

In [24]:
# Leer 150 correos de nuestro conjunto de datos y nos quedamos únicamente con los 50 últimos.
# De estos 50 correos electrónicos no se han utilizado para entrenar el algoritmo.
x, y = create_prep_dataset("datasets/trec07p/full/index",150)
x_test = x[100:]
y_test = y[100:]


Parsing email: 150

##### Preprocesamiento de los correos con el vectorizador creado anteriormente.

In [25]:
x_test = vectorizer.transform(x_test)

##### Predicción del tipo de correo.

In [26]:
y_pred = clf.predict(x_test)
y_pred

array(['spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam'], dtype='<U4')

In [27]:
print("Predicción: \n", y_pred)
print("\nEtiquetas reales:\n", y_test)

Predicción: 
 ['spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam'
 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'ham' 'spam' 'spam' 'spam'
 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'ham' 'spam' 'spam' 'spam'
 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam'
 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam' 'spam']

Etiquetas reales:
 ['spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam', 'spam']


##### Evaluación de Resultados.

In [28]:
from sklearn.metrics import accuracy_score
# Accuracy -> Predicción
print('Accuracy: {:.3f}'.format(accuracy_score(y_test, y_pred)))

Accuracy: 0.940


## 5.- Aumentando el conjunto de datos.

In [29]:
# Leer 12000 correos electrónicos 
x, y = create_prep_dataset("datasets/trec07p/full/index",12000)

Parsing email: 12000

Utilizar 10,000 correos electrónicos para entrenar el algoritmo y 2,000 para realizar pruebas

In [30]:
x_train, y_train = x[:10000], y[:10000]
x_test, y_test = x[10000:], y[10000:]

In [31]:
vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(x_train)

In [32]:
clf = LogisticRegression()
clf.fit(x_train, y_train)

/home/suseth/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [33]:
x_test = vectorizer.transform(x_test)
y_pred = clf.predict(x_test)
print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))

Accuracy: 0.990


In [34]:
# Leer 75419 correos electrónicos 
x, y = create_prep_dataset("datasets/trec07p/full/index",75419)

Parsing email: 75419

In [35]:
x_train, y_train = x[:52793], y[:52793]
x_test, y_test = x[52793:60335], y[52793:60335]

In [36]:
vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(x_train)

In [37]:
clf = LogisticRegression()
clf.fit(x_train, y_train)

/home/suseth/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [38]:
x_test = vectorizer.transform(x_test)
y_pred = clf.predict(x_test)
print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))

Accuracy: 0.986
